## This script downloads a single image, extracts and stores key metadata.

## Dataframes of import

1) df_metadata: captures basic info about the image. image id, dimensions, etc.
2) df_detections: normalized detection coordinates, ready to plot

## Storing results in sqlite

I'm not sure what's the smarter thing to store. df_segments with the base64 encoding, or df_detection_coords.
I elected to go for df_detection_coords as the normalization code is lengthy and confusing. The downside is
you have to remember to serialize the resulting list column before storing and then deserialize after loading.

So either way there's a bit of a non-standard process one has to go through when loading from sqlite


In [9]:
%run set_environment.py

Folder 'D://projects_working_directories//imagery_analysis//20250210_indonesia//' already exists.
Folder 'D://projects_working_directories//imagery_analysis//20250210_indonesia////images' already exists.
Folder 'D://projects_working_directories//imagery_analysis//20250210_indonesia////images_metadata' already exists.
Folder 'D://projects_working_directories//imagery_analysis//20250210_indonesia////output' created successfully.


In [10]:
image_ids = ['817255772895577','935066530801561','1192053898413677']


# Download image and get geometries

In [11]:
import requests
from dotenv import load_dotenv
from os import getenv

from PIL import Image
from io import BytesIO
import pandas as pd
import numpy as np


import sqlite3
import json

import mapillary_utils as mu
import detection_analysis_utils as dau

In [68]:
load_dotenv()
API_KEY = getenv("MAPILLARY_CLIENT_TOKEN")

## vars established by set_environment.py
# base_dir - root directory for working files
# image_folder - fully qualified folder where images are stored
# image_metadata_folder - fully qualified folder where image_metadata is stored
# sqlite3_dbname - fully qualified database file name
# output_folder - default location to write output files

df_metadata = pd.DataFrame(columns=['guid', 'image_source', 'image_id', 'captured_at_unix', 'lat', 'lon',
       'original_height', 'original_width', 'height', 'width', 'camera_type',
       'sequence', 'compass_angle', 'computed_compass_angle', 'is_pano',
       'camera_focal_len', 'camera_k1', 'camera_k2', 'altitude',
       'image_path_on_disk'])
df_detections = pd.DataFrame(columns=['image_id', 'detection_id','detection_label','feature_id','image_height','image_width','extent','properties','coordinates'])

In [76]:
def get_and_process_detections(image_id, API_KEY, image_size_indicator='thumb_original_url', image_dir=image_folder):
    # get the image and metadata
    image, metadata = mu.get_mapillary_image(image_id, API_KEY, image_size_indicator='thumb_original_url', image_dir=image_folder)
    df_metadata = pd.DataFrame.from_dict([metadata])
    
    # get the detections and extract them
    detections = mu.get_mapillary_detections(image_id, API_KEY)
    df_segments = mu.extract_detections(detections)
    
    #merge w/ metadata so can accesss height/width
    df_segments = pd.merge(df_segments,df_metadata, left_on='image_id', right_on='image_id')
    
    # decode detections
    arrays = df_segments.apply(lambda x: mu.decode_base64_geometry_fromdf(x, normalize=True, image_height=x.height, image_width=x.width), axis=1)
    
    df_detections = pd.DataFrame(columns=['image_id', 'detection_id','detection_label','feature_id','image_height','image_width','extent','properties','coordinates'])
    
    #must iterate through like this because for any given detection there can be multiple arrays
    for array in arrays:
        for row in array:
            df_detections.loc[len(df_detections)] = [row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8]]
    
    
    ## get relative pixel count per detection
    df_detections['relative_pixel_count'] = df_detections.coordinates.apply(dau.detect_relative_pixel_count)
    
    # Group by 'image_id' and calculate the sum of 'Value' for each category
    df_detections['relative_image_pixel_count'] = df_detections.groupby('image_id')['relative_pixel_count'].transform('sum')
    df_detections['percent_of_image'] = df_detections.apply((lambda x: (x.relative_pixel_count / x.relative_image_pixel_count)*100 ), axis=1)
    
    return df_metadata, df_detections

for image_id in image_ids:
    temp_meta, temp_detect = get_and_process_detections(image_id, API_KEY, image_size_indicator='thumb_original_url', image_dir=image_folder)
    df_metadata = pd.concat([df_metadata, temp_meta])
    df_detections = pd.concat([df_detections, temp_detect])
    
    #df_detections = pd.concat([df_detections, get_and_process_detections(image_id, API_KEY, image_size_indicator='thumb_original_url', image_dir=image_folder)])
    



C:\Users\ericf\AppData\Local\Temp\ipykernel_5960\1766800250.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_metadata = pd.concat([df_metadata, temp_meta])


In [77]:
df_metadata.to_excel(f"{image_metadata_folder}//sample_image_metadata.xlsx", index=False)

df_detections = df_detections.sort_values(by=['image_id','percent_of_image'], ascending=False)
df_detections.to_excel(f"{image_metadata_folder}//sample_image_detections.xlsx", index=False)

In [85]:
df_metadata[['image_id','lat','lon','camera_type','altitude','image_path_on_disk']]

,image_id,lat,lon,camera_type,altitude,image_path_on_disk
0,817255772895577,3.609915,98.686175,perspective,0,D://projects_working_directories//imagery_anal...
0,935066530801561,3.609505,98.696004,spherical,2.453,D://projects_working_directories//imagery_anal...
0,1192053898413677,3.617158,98.671806,perspective,0,D://projects_working_directories//imagery_anal...
0,817255772895577,3.609915,98.686175,perspective,0,D://projects_working_directories//imagery_anal...
0,935066530801561,3.609505,98.696004,spherical,2.453,D://projects_working_directories//imagery_anal...
0,1192053898413677,3.617158,98.671806,perspective,0,D://projects_working_directories//imagery_anal...


In [90]:
df_detections[['image_id','detection_label','percent_of_image']][(df_detections.image_id == '1192053898413677') & \
    (df_detections.detection_label == 'nature--vegetation')]

,image_id,detection_label,percent_of_image
106,1192053898413677,nature--vegetation,8.558994
124,1192053898413677,nature--vegetation,7.011605
112,1192053898413677,nature--vegetation,1.257253
113,1192053898413677,nature--vegetation,1.015474
107,1192053898413677,nature--vegetation,0.773694
114,1192053898413677,nature--vegetation,0.676983
111,1192053898413677,nature--vegetation,0.483559
108,1192053898413677,nature--vegetation,0.386847
118,1192053898413677,nature--vegetation,0.386847
109,1192053898413677,nature--vegetation,0.338491


In [88]:
df_detections

,image_id,detection_id,detection_label,feature_id,image_height,image_width,extent,properties,coordinates,relative_pixel_count,relative_image_pixel_count,percent_of_image
9,935066530801561,935290274112520,construction--flat--road,1,2880,5760,4096,{},"[[0, 1298], [8, 1295], [16, 1293], [32, 1290],...",307.0,2758.0,11.131255
166,935066530801561,935290710779143,object--wire-group,1,2880,5760,4096,{},"[[1868, 2373], [1895, 2396], [1961, 2430], [19...",136.0,2758.0,4.931109
175,935066530801561,935290717445809,void--ego-vehicle,1,2880,5760,4096,{},"[[0, 849], [153, 855], [288, 852], [340, 840],...",111.0,2758.0,4.024656
57,935066530801561,935290390779175,construction--structure--building,1,2880,5760,4096,{},"[[4418, 1247], [4421, 1252], [4429, 1255], [44...",94.0,2758.0,3.408267
172,935066530801561,935290714112476,object--wire-group,1,2880,5760,4096,{},"[[0, 1892], [112, 1874], [123, 1868], [206, 18...",84.0,2758.0,3.045685
...,...,...,...,...,...,...,...,...,...,...,...,...
289,1192053898413677,1192284248390642,void--unlabeled,1,2160,3840,4096,{},"[[1793, 2138], [1797, 2160], [1831, 2160], [18...",0.0,2068.0,0.000000
291,1192053898413677,1192284255057308,object--traffic-cone,1,2160,3840,4096,{},"[[3744, 335], [3746, 338], [3748, 339], [3757,...",0.0,2068.0,0.000000
292,1192053898413677,1192284258390641,object--support--pole,1,2160,3840,4096,{},"[[568, 146], [573, 162], [575, 184], [589, 182...",0.0,2068.0,0.000000
294,1192053898413677,1192284258390641,object--support--pole,3,2160,3840,4096,{},"[[575, 416], [575, 420], [579, 420], [575, 416]]",0.0,2068.0,0.000000
